# Ownership

Ownership é basicamente o conceito mais importante do rust. É a maneira pela qual o rust implementa a segurança de memória sem a necessidade de um *garbage collector* e sem vulnerabilidades como em linguagens onde o programador é responsável pela alocação de memória como C e C++.

## Mas o que é o Ownership?

Ownership é um conjunto de regras que definem como o programa irá usar a memória do computador. O compilador confere se todas as regras são satisfeitas, caso contrário o programa não compila! Para entender as regras do ownership, antes, devemos ver um pouco sobre as estruturas da memória que o programa irá usar durante o tempo de execução.

# Stack e Heap

A stack e a heap são duas estruturas da memória, disponíveis para o programa durante o runtime. Na stack os itens são guardados na ordem que chegam e removidos na ordem contrária (LIFO). A heap já uma estrutura mais desorganizada, quando algo irá ser colocado na heap, é solicitado um determinado tamanho para aquele dado. O alocador de memória irá então buscar algum espaço que comporte o dado e então retornar um ponteiro para aquela posição. Como o ponteiro é algo de tamanho fixo, ele pode ser guardado na stack. Dessa forma, podemos ver que: Guardar e acessar dados na stack é bem mais rápido que na heap. Quando o programa chama uma função, os argumentos e as variáveis locais da função são colocados na stack (possivelmente, incluindo ponteiros para a heap), assim que a função termina, esses dados são removidos da stack.

# Regras do Ownership

1. Cada valor têm um **"dono"**.
2. Só pode haver um **único** "dono" **por vez**.
3. Uma vez que o "dono" **sair de escopo**, o **valor será droppado**.

## Move Vs Copy

Em resumo, os tipos de dados em rust podem implementar duas *traits* diferentes: **Copy e Drop**. Todos os tipos escalares, arrays e tuplas (cujos elementos também implementem a trait Copy), implementam a trait **Copy**. Isso significa que esses dados são *Stack-only data*, quando alguma variável recebe o seu valor ele é simplesmente copiado na stack e vida que segue. Por isso algo assim funcionaria:

```rust
    let x = 42;
    let y = x; 

    println!("{x}, {y}");
```

Já os tipos complexos, como Strings e Vetores por exemplo, implementam outra trait: **Drop**. E são *Heap-only data*, por isso, quando algo parecido é feito, a propriedade do valor é movida, acontece o **Move**, do valor, e o valor original não pode mais ser acessado:

```rust
    let x = vec![42];
    let y = x;

    println!("{:?}, {:?}", x, y); // error: move occurs because `x` has type `Vec<i32>`, which does not implement the `Copy` trait
```


# Escopo da variável

Como exemplo da 3ª regra de Ownership, note que:

In [ ]:
let x = vec![1];
let y = x;

println!("{:?}, {:?}", x, y);

In [ ]:
{
    let s: &str = "Hello, World!";
    println!("{}", s);
}
// println!("{}", s); // error: cannot find value `s` in this scope

No código acima, a variável `s` é válida, enquanto o escopo dela existe, após o fim deste escopo (representado pelas `{}`) o rust automaticamente chama a função `drop()`, que irá remover `s` da stack. O tipo da variável é uma **string slice** (`&str`), que é imutável e de tamanho fixo, diferente do tipo `String`. Por isso, ele pode ser guardado na stack e facilmente copiado. Isso faz com que o código abaixo seja possível.

In [ ]:
let s1 = "Hello";
let s2 = nome;
println!("S1: {s1}, S2: {s2}");

No entanto, quando for necessário usar o tipo `String`, deve ser chamada a função `from()` do tipo `String`. Como este tipo é mutável e aumentável, ao chamar esta função o alocador de memória irá separar um espaço na heap para a String. E o que será guardado na stack são três valores: Um ponteiro para a String na heap, seu tamanho e sua capacidade. Algo parecido com a ilustração abaixo:

<center>

```mermaid
    graph LR;
        subgraph "Stack";
            A["ptr da s1"]
        end 

        subgraph "Heap";
            D["Hello"]
        end

        A --> D;
```

</center>

Por isso ao tentar fazer algo como:

In [ ]:
let s1 = String::from("Hello");
let s2 = s1;
println!("{s1}, world!"); // error: value borrowed here after move

Não será possível, o compilador não deixará. Fazer isso seria o equivalente a ter dois ponteiros na stack para o mesmo endereço da heap. Isso resulta em um erro já conhecido e bastante comum, o *double* *free* *error*.

<center>

```mermaid
    graph LR;
        subgraph "Stack";
            A["ptr da s1"];

            A1["ptr da s2"];    
        end 

        subgraph "Heap";
            D["Hello"]
        end

        A --> D;
        A1 --> D;
```

</center>

Ao invés disso, se é desejado explicitamente copiar os dados de `s1`, pode ser usado o método `clone()`, que irá resultar em algo do tipo, onde o valor da heap também será copiado:


<center>

```mermaid
    graph LR;
    subgraph Stack
        A["ptr da s1"]
        A1["ptr da s2"]
    end

    subgraph Heap
        D["Hello"]
        D1["Hello"]
    end

    A --> D
    A1 --> D1
```

</center>

In [ ]:
let s1 = String::from("Hello");
let s2 = s1.clone();
println!("{s1}, world!"); // ok

In [ ]:
fn greeting(s: String){
    println!("Hello, {}!", s);
}

fn main() {
    let s = String::from("Guilherme");
    greeting(s);

    println!("{}", s); // error: value borrowed here after move
}

Ao ler a mensagem de erro, entende-se que a posse da string `s` foi transferida para `greeting()`, por isso, quando o escopo da função termina, o valor de `s` é droppado, e não pode ser acessado de novo. Uma maneira de resolver isso seria retornar o valor de `s` para a variável novamente, dessa forma a posse seria transferida antes que o escopo terminasse:

In [ ]:
fn greeting(s: String) -> String{
    println!("Hello, {}!", s);
    s
}

fn main() {
    let mut s = String::from("Guilherme");
    s = greeting(s);

    println!("{}", s); 
}
main();

No entanto, é um tanto quanto inconveniente ter que retornar um valor toda vez que você deseja usá-lo novamente. Como fazer isso sem retornar o valor novamente?

# Borrowing e Referências

Devido a essa mecânica de "donos", para que seja possível passar variáveis como argumentos, ao invés de mover a posse de um dado para outro dono, é possível simplesmente **"emprestar"** (borrow) um valor, passando uma **referência** ao valor, ao invés do valor de fato. Uma referência pode ser entendida como um ponteiro, porém ao contrário dos ponteiros convencionais, é **garantido** que uma referência sempre apontará para um valor válido. Em rust, não existe o conceito de `null`, ao invés disso a ausência de valores é tratada de outra maneira que veremos posteriormente. Uma referência é passada usando o operador `&`. A função anterior ficaria da seguinte forma:

In [ ]:
fn greeting(s: &String) {
    println!("Hello, {}!", s);
}

fn main() {
    let mut s = String::from("Guilherme");
    greeting(&s);

    println!("{}", s); 
}
main();

Dessa forma, a função `greeting()` não ganha a posse do valor de `s`, e assim, `s` pode ser acessado novamente depois. Uma referência pode ser entendida, ao menos é a forma mais fácil que consegui explicar, como um **ponteiro para outro ponteiro**. Na stack seria algo na linha de:

<center>

```mermaid
    graph LR;
    subgraph Stack
        A["ptr de s"]
        A1["ref para s"]
    end

    subgraph Heap
        D["Hello"]
    end

    A1 --> A
    A --> D
```

</center>

> Nota: O operador contrário de `&` é o `*` (deferência).

# Referências mutáveis

Por padrão, uma referência é imutável, ou seja, o valor para o qual ela aponta não pode ser modificado. Como mostra o código abaixo:

In [ ]:
fn add_world(s: &String) {
    s.push_str(", World!");
}

fn main() {
    let mut s = String::from("Hello");

    add_world(&s); // error: cannot borrow `s` as mutable, as it is not declared as mutable
}

main();

Isso pode ser corrigido simplesmente passando uma referência mutável para a função.

In [ ]:
fn add_world(s: &mut String) {
    s.push_str(", World!");
}

fn main() {
    let mut s = String::from("Hello");

    add_world(&mut s); 

    println!("{}", s);
}

main();

> #### Referências mutáveis possuem uma única grande restrição, se há uma referência mutável, ela deve ser a única para um objeto por escopo.

In [ ]:
let mut s = String::from("Hello");

let r1 = &mut s;
let r2 = &mut s;

println!("{} {}", r1, r2); // error: cannot borrow `s` as mutable more than once at a time


In [ ]:

let mut s = String::from("Hello");

{
    let r1 = &mut s; // ok
}

let r2 = &mut s;

Contudo, não há restrições para referências imutáveis, elas podem ser ilimitadas. Apenas se houver alguma **mutável**, que essa restrição é válida.

In [ ]:
fn main() {
    let mut s = String::from("Hello");

    let r1 = &s;
    let r2 = &s;
    println!("{r1} and {r2}");  // ok

    let r3 = &mut s;
    println!("{r3}"); // ok, the both r1 and r2 were already dropped
}

main();

## Regras das referências

1. Por escopo, deve haver ou uma única referência mutável ou qualquer número de imutáveis.
2. Referências sempre devem ser válidas.

# Slices

Slices são um tipo especial de referências, são referências para uma parte contígua de uma coleção, não necessariamente a coleção inteira. Elas são úteis para permitir o acesso seguro e eficiente a subseções de dados, sem a necessidade de copiar esses dados. 

In [ ]:
fn main() {
    let arr = [1, 2, 3, 4, 5];
    
    // Slice imutável
    let slice = &arr[1..4]; // [2, 3, 4]
    println!("{:?}", slice);

    // Slice mutável
    let mut vec = vec![10, 20, 30, 40, 50];
    let slice_mut = &mut vec[2..4]; // [30, 40]
    slice_mut[0] = 35;  // Modifica o primeiro elemento da slice mutável
    println!("{:?}", slice_mut); // [35, 40]
}

main();

String literals são tratadas como **string slices** `&str`.